
# Telecom X — Parte 2: Predicción de Cancelación (Churn)
Este notebook construye un pipeline de ML para predecir la evasión de clientes. Incluye preparación, modelado, evaluación y reporte.


In [ ]:

import os, sys, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importar utilidades
ROOT = Path.cwd().parents[1] if (Path.cwd().name == "notebooks") else Path.cwd()
sys.path.append(str(ROOT/"src"))
from prep import load_clean_csv, drop_id_columns, split_features_target, get_cat_num_cols
from models import train_evaluate_models

FIGS = ROOT/"figs"; FIGS.mkdir(exist_ok=True, parents=True)
REPORT = ROOT/"reporte"; REPORT.mkdir(exist_ok=True, parents=True)

# Ajustes de visualización (sin estilos especiales)
plt.rcParams["figure.figsize"] = (9,5)
plt.rcParams["axes.grid"] = True


In [ ]:

# === Cargar datos (CSV limpio de la Parte 1) ===
CSV_PATH = ROOT / "data" / "TelecomX_Data_clean.csv"  # <--- coloca aquí tu archivo
if not CSV_PATH.exists():
    print("⚠️ No se encontró el CSV:", CSV_PATH)
    print("Coloca tu CSV limpio en la carpeta 'data/' y vuelve a ejecutar esta celda.")
else:
    df = load_clean_csv(str(CSV_PATH))
    print("Datos cargados:", df.shape)
    display(df.head(3))


In [ ]:

# === Preprocesamiento: eliminar ID, verificar target y tipos ===
if 'df' in globals():
    df = drop_id_columns(df)
    if "Churn" not in df.columns:
        raise ValueError("No se encontró la columna 'Churn' en el CSV limpio.")
    # Asegurar target binario 0/1
    if df["Churn"].dtype != int and df["Churn"].dtype != "int64":
        df["Churn"] = df["Churn"].astype(int)
    # Info general
    display(df.info())
    display(df.describe(include="all").T.head(15))


In [ ]:

# === Balance de clases ===
if 'df' in globals():
    churn_counts = df["Churn"].value_counts().sort_index()
    print("Distribución Churn (0=no, 1=sí):")
    display(churn_counts)
    plt.figure()
    churn_counts.plot(kind="bar")
    plt.title("Distribución de Churn")
    plt.xlabel("Churn")
    plt.ylabel("Conteo")
    plt.tight_layout()
    plt.savefig(FIGS/"01_distribucion_churn.png", dpi=160)
    plt.show()

    ratio = churn_counts.min() / churn_counts.max() if len(churn_counts)>1 else 1.0
    print(f"Ratio clase minoritaria/mayoritaria: {ratio:.3f}")


In [ ]:

# === Correlación numérica con Churn ===
if 'df' in globals():
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    corr = df[num_cols].corr()
    plt.figure()
    plt.imshow(corr, interpolation='nearest')
    plt.title("Matriz de correlación (numéricas)")
    plt.colorbar()
    plt.xticks(range(len(num_cols)), num_cols, rotation=90)
    plt.yticks(range(len(num_cols)), num_cols)
    plt.tight_layout()
    plt.savefig(FIGS/"02_matriz_correlacion.png", dpi=160)
    plt.show()

    if "tenure" in df.columns:
        plt.figure()
        df.boxplot(column="tenure", by="Churn")
        plt.title("tenure vs Churn"); plt.suptitle("")
        plt.tight_layout(); plt.savefig(FIGS/"03_box_tenure_churn.png", dpi=160); plt.show()

    if "TotalCharges" in df.columns:
        plt.figure()
        df.boxplot(column="TotalCharges", by="Churn")
        plt.title("TotalCharges vs Churn"); plt.suptitle("")
        plt.tight_layout(); plt.savefig(FIGS/"04_box_totalcharges_churn.png", dpi=160); plt.show()


In [ ]:

# === Preparar X, y y entrenar múltiples modelos ===
if 'df' in globals():
    X, y = split_features_target(df, target_col="Churn")
    # ¿Aplicar SMOTE? Cámbialo a True si está muy desbalanceado y tienes imbalanced-learn instalado
    USE_SMOTE = False
    out = train_evaluate_models(X, y, test_size=0.3, random_state=42, use_smote=USE_SMOTE)
    results = out["results"]
    print("Métricas por modelo:")
    display(pd.DataFrame(results).T)


In [ ]:

# === Matriz de confusión para el mejor modelo (por F1) ===
if 'df' in globals():
    # Elegir mejor por F1
    res_df = pd.DataFrame(results).T
    best_model_name = res_df["f1"].idxmax()
    print("Mejor modelo por F1:", best_model_name)

    from sklearn.metrics import ConfusionMatrixDisplay
    # Necesitamos regenerar predicciones para graficar
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    model = out["models"][best_model_name]
    y_pred = model.predict(X_test)
    disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
    plt.title(f"Matriz de confusión — {best_model_name}")
    plt.tight_layout(); plt.savefig(FIGS/"05_confusion_matrix.png", dpi=160); plt.show()


In [ ]:

# === Importancia / Coeficientes de variables ===
if 'df' in globals():
    best_model_name = pd.DataFrame(results).T["f1"].idxmax()
    model = out["models"][best_model_name]
    feat_names = out["feature_names"].get(best_model_name, None)

    # Coeficientes (Logistic Regression) o Importancias (RandomForest)
    try:
        if hasattr(model.named_steps["clf"], "coef_"):
            coefs = model.named_steps["clf"].coef_[0]
            if feat_names is not None and len(feat_names)==len(coefs):
                imp_df = pd.DataFrame({"feature": feat_names, "importance": np.abs(coefs)}).sort_values("importance", ascending=False).head(15)
            else:
                imp_df = pd.DataFrame({"importance": np.abs(coefs)}).head(15)
        elif hasattr(model.named_steps["clf"], "feature_importances_"):
            imps = model.named_steps["clf"].feature_importances_
            if feat_names is not None and len(feat_names)==len(imps):
                imp_df = pd.DataFrame({"feature": feat_names, "importance": imps}).sort_values("importance", ascending=False).head(15)
            else:
                imp_df = pd.DataFrame({"importance": imps}).head(15)
        else:
            imp_df = pd.DataFrame({"mensaje": ["El modelo no expone importancias directas (p.ej., KNN)."]})
    except Exception as e:
        imp_df = pd.DataFrame({"error": [str(e)]})

    display(imp_df)

    # Gráfico simple de importancias si existen
    if "feature" in imp_df.columns and "importance" in imp_df.columns:
        plt.figure()
        imp_df.iloc[::-1].plot(kind="barh", x="feature", y="importance", legend=False)
        plt.title(f"Top características — {best_model_name}")
        plt.tight_layout(); plt.savefig(FIGS/"06_top_importancias.png", dpi=160); plt.show()


In [ ]:

# === Generar informe de modelado ===
if 'df' in globals():
    lines = []
    lines.append("# Informe de Modelado — Churn en Telecom X\n")
    lines.append("## Objetivo\nPredecir clientes con mayor probabilidad de cancelar (Churn) usando modelos de clasificación.\n")
    lines.append("## Preparación de datos\n- Eliminación de IDs.\n- Conversión de `Churn` a binario.\n- One-Hot Encoding para categóricas y escalado para modelos sensibles.\n")
    lines.append("## Distribución de clases y correlaciones\nSe adjuntan gráficos en `figs/`.\n")
    lines.append("## Modelos y métricas\n")
    lines.append(pd.DataFrame(results).T.to_markdown())
    lines.append("\n")
    lines.append(f"**Mejor modelo (F1):** {pd.DataFrame(results).T['f1'].idxmax()}\n")
    lines.append("## Importancia de variables\nRevisar el gráfico de importancias (si aplica) en `figs/06_top_importancias.png`.\n")
    lines.append("## Conclusiones e Insights\n- Contratos mensuales, mayor gasto y baja permanencia suelen asociarse con más churn.\n- Combinar ofertas y fidelización para clientes de alto riesgo.\n")
    lines.append("## Próximos pasos\n- Ajuste de hiperparámetros (GridSearch/RandomizedSearch).\n- Validación cruzada y monitoreo en producción.\n")

    out = REPORT / "informe_modelado.md"
    with open(out, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    print("Informe guardado en:", out)
